In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import argparse
import torch
import re
from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer
from model.net import KobertSequenceFeatureExtractor, KobertCRF, KobertBiLSTMCRF, KobertBiGRUCRF
from gluonnlp.data import SentencepieceTokenizer
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

import numpy as np
import itertools

from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from krwordrank.word import summarize_with_keywords

/opt/anaconda3/envs/kobert_ner/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
hannanum = Hannanum()
#한국어 불용어
stop_words="아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓"
stop_words=stop_words.split(' ')

# SBERT를 로드
sbert_model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [3]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        print("len: {}, input_token:{}".format(len(input_token), input_token))
        print("len: {}, pred_ner_tag:{}".format(len(pred_ner_tag), pred_ner_tag))

        # ----------------------------- parsing list_of_ner_word ----------------------------- #
        list_of_ner_word = []
        entity_word, entity_tag, prev_entity_tag = "", "", ""
        for i, pred_ner_tag_str in enumerate(pred_ner_tag):
            if "B-" in pred_ner_tag_str:
                entity_tag = pred_ner_tag_str[-3:]

                if prev_entity_tag != entity_tag and prev_entity_tag != "":
                    list_of_ner_word.append({"word": entity_word.replace("▁", " "), "tag": prev_entity_tag, "prob": None})

                entity_word = input_token[i]
                prev_entity_tag = entity_tag
            elif "I-"+entity_tag in pred_ner_tag_str:
                entity_word += input_token[i]
            else:
                if entity_word != "" and entity_tag != "":
                    list_of_ner_word.append({"word":entity_word.replace("▁", " "), "tag":entity_tag, "prob":None})
                entity_word, entity_tag, prev_entity_tag = "", "", ""


        # ----------------------------- parsing decoding_ner_sentence ----------------------------- #
        decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False

        for token_str, pred_ner_tag_str in zip(input_token, pred_ner_tag):
            token_str = token_str.replace('▁', ' ')  # '▁' 토큰을 띄어쓰기로 교체

            if 'B-' in pred_ner_tag_str:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'

                if token_str[0] == ' ':
                    token_str = list(token_str)
                    token_str[0] = ' <'
                    token_str = ''.join(token_str)
                    decoding_ner_sentence += token_str
                else:
                    decoding_ner_sentence += '<' + token_str
                is_prev_entity = True
                prev_entity_tag = pred_ner_tag_str[-3:] # 첫번째 예측을 기준으로 하겠음
                is_there_B_before_I = True

            elif 'I-' in pred_ner_tag_str:
                decoding_ner_sentence += token_str

                if is_there_B_before_I is True: # I가 나오기전에 B가 있어야하도록 체크
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token_str
                    is_prev_entity = False
                    is_there_B_before_I = False
                else:
                    decoding_ner_sentence += token_str

        return list_of_ner_word, decoding_ner_sentence

In [4]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [5]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [6]:
#parser, ner_model경로 설정

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', default='./data_in', help="Directory containing config.json of data")
parser.add_argument('--model_dir', default='./experiments/base_model_with_crf_val', help="Directory containing config.json of model")
args, _ = parser.parse_known_args()
ner_model_dir = Path(args.model_dir)
ner_model_config = Config(json_path=ner_model_dir / 'config.json')

In [7]:
# Vocab & Tokenizer
# tok_path = get_tokenizer() # ./tokenizer_78b3253a26.model
tok_path = "./ptr_lm_model/tokenizer_78b3253a26.model"
ptr_tokenizer = SentencepieceTokenizer(tok_path)

# load vocab & tokenizer
with open(ner_model_dir / "vocab.pkl", 'rb') as f:
    vocab = pickle.load(f)

tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=ner_model_config.maxlen)

# load ner_to_index.json
with open(ner_model_dir / "ner_to_index.json", 'rb') as f:
    ner_to_index = json.load(f)
    index_to_ner = {v: k for k, v in ner_to_index.items()}

ner_model = KobertCRF(config=ner_model_config, num_classes=len(ner_to_index), vocab=vocab)

# load
ner_model_dict = ner_model.state_dict()
checkpoint = torch.load("./experiments/base_model_with_crf_val/best-epoch-9-step-750-acc-0.957.bin", map_location=torch.device('cpu'))

convert_keys = {}
for k, v in checkpoint['model_state_dict'].items():
    new_key_name = k.replace("module.", '')
    if new_key_name not in ner_model_dict:
        print("{} is not int model_dict".format(new_key_name))
        continue
    convert_keys[new_key_name] = v

ner_model.load_state_dict(convert_keys)
ner_model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# n_gpu = torch.cuda.device_count()
# if n_gpu > 1:
#     model = torch.nn.DataParallel(model)
ner_model.to(device)

decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)

In [8]:
doc_with_newlines = """
비대면의 일상화와 기업들의 디지털 전환 등으로 정보 보안의 중요성이 커지고 있다. 지난해 애플, 구글, 마이크로소프트 등 빅테크기업이 사이버보안기업에 투자한 금액만 23억9700만달러(약 3조원)에 이른다. 이에 비해 한국은 자본금 50억원 미만 기업이 93%(494개)를 차지할 정도로 보안산업이 홀대받고 있다.

 ○확산하는 사이버테러 공포

11일 업계에 따르면 정보보안 위협이 심각해지면서 각국 정부는 물론 기업들도 보안을 강화하는 추세다. 러시아가 전쟁에 앞서 우크라이나 정부 전산망과 금융회사에 대규모 사이버 공격을 하면서 사이버전쟁에 대한 공포가 커졌다는 분석이다. 최근 해커그룹 랩서스가 삼성전자와 LG전자, 마이크로소프트, 엔비디아 등 글로벌 기업의 계정 정보와 소스코드를 탈취하는 등 기업 관련 보안 사고도 꾸준히 늘고 있다.

글로벌 기업들은 보안 강화를 위해 과감히 ‘지갑’을 열고 있다. 구글은 지난 3월 사이버보안 전문기업 맨디언트를 54억달러(약 6조8000억원)에 인수했다. 2012년 모토로라 모빌리티(125억달러) 이후 가장 투자 규모가 컸던 인수합병(M&A)이다. 맨디언트는 정보 침해 대응 전문기업으로 보안 사고가 일어나면 해킹의 원인과 보완점 등을 알려준다. 마이크로소프트 역시 지난해 7월 클라우드녹스시큐리티와 리스크IQ를 잇달아 인수하면서 클라우드 보안을 강화했다.

미국 시장조사업체 CB인사이트에 따르면 구글, 애플, 마이크로소프트 등 빅테크기업의 지난해 사이버보안기업 투자금액은 역대 최고치인 23억9700만달러에 달했다. 직전 5년(2016~2020년) 투자 금액과 맞먹는 금액이다. 가트너에 따르면 글로벌 정보보안 시장은 2020년 1319억달러(약 168조원)에서 2024년 1887억달러(약 240조400억원)까지 성장할 전망이다.

 ○걸음마 단계에 머무는 한국

국내 보안 시장은 여전히 걸음마 단계다. 한국 정보보안기업의 2020년 총매출은 3조9213억원이다. 2014년부터 연평균 14.5% 성장했으며 이 분야 기업 수는 2020년 기준 531곳이다. 하지만 이 가운데 종사자 20인 미만 기업이 41.6%(218곳)를 차지한다. 200인 이상 기업은 33곳에 불과하다.

국내에서 시가총액이 1조원을 넘는 곳은 안랩뿐이다. 11일 기준 1조1000억원 수준이다. 이외 주요 기업은 윈스(2341억원) 라온시큐어(1290억원) 파수(1198억원) 등이다. 안랩은 창업자인 안철수 전 대통령직인수위원장의 행보에 따라 주가가 급등락을 반복하고 있다. 보안업계 한 관계자는 “국내 최대 보안기업이 ‘정치인 테마주’로 분류되는 게 한국 정보보안산업의 현실”이라고 지적했다.

지난 6일에는 SK쉴더스가 기업공개(IPO)를 철회하기도 했다. 이 회사는 SK스퀘어 자회사인 정보보안기업 SK인포섹과 물리보안기업 ADT캡스를 통합해 출범한 융합보안기업이다. 지난해 정보보안 매출이 3381억원으로 안랩(2072억원)보다 많다. 하지만 기관투자가 대상 수요예측 결과가 부진해 상장을 포기했다.

임종인 고려대 정보보호대학원 석좌교수는 “정부 주도로 인재를 양성해 이들이 스타트업을 설립하고 해외 시장으로 진출해 선순환 구조를 형성한 이스라엘의 사례를 참고할 만하다”며 “해외 시장 문을 두드릴 수 있는 인재를 키우는 데 초점을 맞춰야 한다”고 강조했다.

"""
doc=doc_with_newlines.replace(".", "\n")
#한글, 영어, 숫자, +, -, 개행, 띄어쓰기만 놔두고 다 제거
doc = re.sub(r"[^가-힣a-zA-Z0-9\+\-\n\s]","",doc)

#문장을 분리하였지만, 아직 앞뒤 공백이 있음
temp_texts =  doc.split('\n')
#앞뒤 공백이 제거된 문자열들이 저장되는 list
texts=[]
for text in temp_texts:
    text = text.strip()
    if text is not '':
        texts.append(text)

print(texts)

['비대면의 일상화와 기업들의 디지털 전환 등으로 정보 보안의 중요성이 커지고 있다', '지난해 애플 구글 마이크로소프트 등 빅테크기업이 사이버보안기업에 투자한 금액만 23억9700만달러약 3조원에 이른다', '이에 비해 한국은 자본금 50억원 미만 기업이 93494개를 차지할 정도로 보안산업이 홀대받고 있다', '확산하는 사이버테러 공포', '11일 업계에 따르면 정보보안 위협이 심각해지면서 각국 정부는 물론 기업들도 보안을 강화하는 추세다', '러시아가 전쟁에 앞서 우크라이나 정부 전산망과 금융회사에 대규모 사이버 공격을 하면서 사이버전쟁에 대한 공포가 커졌다는 분석이다', '최근 해커그룹 랩서스가 삼성전자와 LG전자 마이크로소프트 엔비디아 등 글로벌 기업의 계정 정보와 소스코드를 탈취하는 등 기업 관련 보안 사고도 꾸준히 늘고 있다', '글로벌 기업들은 보안 강화를 위해 과감히 지갑을 열고 있다', '구글은 지난 3월 사이버보안 전문기업 맨디언트를 54억달러약 6조8000억원에 인수했다', '2012년 모토로라 모빌리티125억달러 이후 가장 투자 규모가 컸던 인수합병MA이다', '맨디언트는 정보 침해 대응 전문기업으로 보안 사고가 일어나면 해킹의 원인과 보완점 등을 알려준다', '마이크로소프트 역시 지난해 7월 클라우드녹스시큐리티와 리스크IQ를 잇달아 인수하면서 클라우드 보안을 강화했다', '미국 시장조사업체 CB인사이트에 따르면 구글 애플 마이크로소프트 등 빅테크기업의 지난해 사이버보안기업 투자금액은 역대 최고치인 23억9700만달러에 달했다', '직전 5년20162020년 투자 금액과 맞먹는 금액이다', '가트너에 따르면 글로벌 정보보안 시장은 2020년 1319억달러약 168조원에서 2024년 1887억달러약 240조400억원까지 성장할 전망이다', '걸음마 단계에 머무는 한국', '국내 보안 시장은 여전히 걸음마 단계다', '한국 정보보안기업의 2020년 총매출은 3조9213억원이다', '2014년부터 연평균 14', '5 성장했으며 이 분야 기업

In [9]:
# 품사 태깅
tokenized_doc = hannanum.pos(doc)
#print(tokenized_doc)
# 형태소 분석기를 통해 명사 추출한 문서를 만듦
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc
                   if (word[1] == 'N' or word[1] == 'F')
                   and not word[0].isdigit() 
                   and len(word[0])>1
                   and word[0] not in stop_words
                   and word[0].isalnum()])

n_gram_range = (1, 1)
# CountVectorizer를 사용하여 n-gram을 추출(n_gram_range에 해당하는)
count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names()

#doc를 임베딩
doc_embedding = sbert_model.encode([doc])
#text들을 임베딩
texts_embedding = sbert_model.encode(texts)
#n_gram들을 임베딩
candidate_embeddings = sbert_model.encode(candidates)

In [10]:
org = set([])
per = set([])
poh = set([])

for text in texts:
    list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([text])
    ner_model_input = torch.tensor(list_of_input_ids).long()

    ## for bert alone
    # y_pred = model(x_input)
    # list_of_pred_ids = y_pred.max(dim=-1)[1].tolist()

    ## for bert crf
    list_of_pred_ids = ner_model(ner_model_input)

    list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
    print("decoding_ner_sentence:", decoding_ner_sentence)
    print("list_of_ner_word", list_of_ner_word)
    
    for ner_word in list_of_ner_word:
        tag=ner_word['tag']
        word = ner_word['word'].strip()
        if(tag == 'ORG' and len(word)>1):
            org.add(word)
        if(tag == 'PER' and len(word)>1):
            per.add(word)
        if(tag == 'POH' and len(word)>1):
            poh.add(word)

/Users/baeseonghyun/Desktop/keyword/pytorch-crf/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_croot-udngs7fm/pytorch_1648016055234/work/aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


len: 21, input_token:['[CLS]', '▁비', '대', '면', '의', '▁일상', '화', '와', '▁기업들', '의', '▁디지털', '▁전환', '▁등으로', '▁정보', '▁보안', '의', '▁중요성', '이', '▁커지고', '▁있다', '[SEP]']
len: 21, pred_ner_tag:['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 비대면의 일상화와 기업들의 디지털 전환 등으로 정보 보안의 중요성이 커지고 있다[SEP]
list_of_ner_word []
len: 34, input_token:['[CLS]', '▁지난해', '▁애플', '▁구글', '▁마', '이', '크', '로', '소프트', '▁등', '▁빅', '테크', '기업', '이', '▁사이버', '보', '안', '기업', '에', '▁투자', '한', '▁금액', '만', '▁23', '억', '9', '700', '만달러', '약', '▁3', '조원', '에', '▁이른다', '[SEP]']
len: 34, pred_ner_tag:['[CLS]', 'O', 'B-ORG', 'B-ORG', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MNY', 'I-MNY', 'I-MNY', 'I-MNY', 'I-MNY', 'O', 'B-MNY', 'I-MNY', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] 지난해 <애플:ORG> <구글:ORG> <마이크로소프트:ORG> 등 빅테크기업이 사이버보안기업에 투자한 금액만 <23억9700만달러:MNY>약 <3조원:

len: 40, input_token:['[CLS]', '▁', '가', '트', '너', '에', '▁따르면', '▁글로벌', '▁정보', '보', '안', '▁시장은', '▁20', '20', '년', '▁13', '19', '억달러', '약', '▁1', '68', '조원', '에서', '▁20', '24', '년', '▁18', '87', '억달러', '약', '▁2', '40', '조', '400', '억원', '까지', '▁성장', '할', '▁전망이다', '[SEP]']
len: 40, pred_ner_tag:['[CLS]', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'B-MNY', 'I-MNY', 'I-MNY', 'O', 'B-MNY', 'I-MNY', 'I-MNY', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'B-MNY', 'I-MNY', 'I-MNY', 'O', 'B-MNY', 'I-MNY', 'I-MNY', 'I-MNY', 'I-MNY', 'O', 'O', 'O', 'O', '[SEP]']
decoding_ner_sentence: [CLS] <가트너:ORG>에 따르면 글로벌 정보보안 시장은 <2020년:DAT> <1319억달러:MNY>약 <168조원:MNY>에서 <2024년:DAT> <1887억달러:MNY>약 <240조400억원:MNY>까지 성장할 전망이다[SEP]
list_of_ner_word [{'word': ' 가트너', 'tag': 'ORG', 'prob': None}, {'word': ' 2020년', 'tag': 'DAT', 'prob': None}, {'word': ' 1319억달러', 'tag': 'MNY', 'prob': None}, {'word': ' 168조원', 'tag': 'MNY', 'prob': None}, {'word': ' 2024년', 'tag': 'DAT', 

In [11]:
print("org: ", org)
print("per: ", per)
print("poh: ", poh)

org:  {'한국', 'SK스퀘어', '모토로라 모빌리티', '맨디언트', 'SK쉴더스가', 'ADT캡스를', '구글', '러시아', 'CB인사이트', '우크라이나', '한국은', '마이크로소프트', '윈스', 'SK인포섹', '고려대 정보보호대학원', '리스크IQ', '삼성전자', '엔비디아', '안랩', '이스라엘', '파수', '미국', '클라우드녹스시큐리티', '가트너', '라온시큐어'}
per:  {'안철수', '임종인'}
poh:  set()


In [12]:
# nr_candidate가 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
final_keywords=max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)
print(final_keywords)

# diversity값이 클수록, 키워드간의 유사성이 최소화 됨, 문서와 키워드 사이의 유사성은 극대화
final_keywords=mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)
print(final_keywords)

['보안기업', '보안업계', '해커그룹', '빅테크기업', '사이버전쟁']
['사이버보안기업', '보안산업', '빅테크기업', '사이버전쟁', '정보보안산업']
